$\textbf{GOAL}$ Create from scratch a fully connected neural network (ANN) and get exact result than library already integrated.

**This notebook is nota at all the main subject of the intership, more and better explaination exists other than here. Its more a compilation of ideas on neural network and code test than a explicit course.** 

<span style="color:red"><b>TODO :</b></span> Je pense un probleme sur la matrice de poids du from scratch. 

On doit tout reprendre proprement avec des tests solides.

Pas assez convergent sur le cas linéaire. 
Peut-être rester sur du Lux et après voir pourquoi le from scratch ne fonctionne pas.

In [1]:
using LinearAlgebra, Random, Printf

In [2]:
include("../functions/ISTA.jl")

import .ISTA: ista_L, ista

In [3]:
struct Dense_scratch
    W::Matrix{Float64}
    b::Vector{Float64}
    ς::Function
    ς′::Function
end

mutable struct MLP
    layers::Vector{Dense_scratch}
end

# Theory

## Basis

We need ANN because it allows to modelize non linear function from ther activation function which are non linear.

**Definition.**
For an input $x\in\mathbb R^{p_1}$ and an integer $l\ge 1$ (total layers), a *fully-connected ANN* is the composite map

$$
\mu_\theta(x) =S_l\circ S_{l-1}\circ\cdots\circ S_{1}(x)
$$

with parameters
$$
\theta = \bigl(W_1, (W_2,\dots,W_l,b_1, \dots, b_l)\bigr) =(\theta^{(1)},\theta^{(2)})$$

where, for $k=\llbracket 1, l-1 \rrbracket$
$$
S_k(u)=\varsigma\!\bigl(b_k+W_k u\bigr),
\qquad
S_l(u)=b_l+W_l u,
$$

$W_k\in\mathbb R^{p_{k+1}\times p_k}$, $b_k\in\mathbb R^{p_{k+1}}$, and $p_{l+1}=1$ for scalar regression ($b_l = c$ the intercept)

From ``lambda_qut.ipynb`` we need assumptions.  
<span style="color:red"><b>TODO :</b></span>  Les hypotheses sont plausibles mais pas sûr de ce que j'avance.  
To keep the zero-function reachable and allow QUT-compatible sparsity we impose 
$\varsigma\in C^2$, unbounded, $L$-Lipschitz, $\varsigma(0)=0$ and $\varsigma'(0)>0$ like shifted-ReLU ($ReLU_\alpha$), $ELU_\alpha$, Softplus.

## Back Propagation

### Chain rule

Given a loss function we need to minimize the error made by the learner $\mu$, so we will optimize it by **BackPropagation**

On theLASSO problem (from chatGPT), for any weight $W_k$,
$$
\partial_{W_k} \tfrac12\|y-\mu_\theta(X)\|_2^2
=
\Bigl(J_{S_{k+1:l}}(z_k)^\top\bigl(\mu_\theta(X)-y\bigr)\Bigr)z_{k-1}^\top,
$$

where $z_{k-1}=S_{k-1}\circ\cdots\circ S_1(X)$ and $J_{S_{k+1:l}}$ is the Jacobian of the suffix $S_l\circ\cdots\circ S_{k+1}$. This is the usual chain-rule implemented by back-prop.



I've written this few years ago, it can be helpful.  
Let $n\in \mathbb{N}^*$ be the sample size of data, $X_i \in \mathbb{R}^{N_0}$ be one input data and $y_i, \hat{y}_i \in \mathbb{R}^{N_L}$ be, respectively, the true output associated with $X_i$ and the prediction inferred by the model (in this case, the perceptron) to the $i$-th data element. 



We have via the chain rule,
$$\frac{\partial L}{\partial w_{ij}} = \frac{\partial L}{\partial \hat{y}_i} \cdot \frac{\partial \hat{y}_i}{\partial Z_i} \cdot \frac{\partial Z_i}{\partial w_{ij}}$$  


We can rewrite it to get a general code later, for one parameter block the full chain rule by fixing a layer index $1\le \ell\le L$.
The loss is $\mathcal L\bigl(a^{(L)},y\bigr)$ with $y$ the target.
$$
\frac{\partial\mathcal{L}}{\partial W^{(\ell)}} 
= 
\frac{\partial\mathcal{L}}{\partial a^{(L)}}
\cdot \frac{\partial a^{(L)}}{\partial z^{(L)}}
\cdot \frac{\partial z^{(L)}}{\partial a^{(L-1)}}
\cdots
\frac{\partial a^{(\ell)}}{\partial z^{(\ell)}}
\cdot \frac{\partial z^{(\ell)}}{\partial W^{(\ell)}}
$$

Define the error signal,
$$
\delta^{(\ell)} := \frac{\partial \mathcal L}{\partial z^{(\ell)}} \in \mathbb{R}^{n_\ell}.
$$

By factorising the long product above, we obtain the recursive form,
$$
\delta^{(L)} = \frac{\partial \mathcal{L}}{\partial a^{(L)}} 
\odot \sigma^{(L)\prime}\big(z^{(L)}\big)
$$

$$\delta^{(\ell)} = \bigl(W^{(\ell+1)}\bigr)^\top \delta^{(\ell+1)}
               \odot \sigma^{(\ell)\prime}\bigl(z^{(\ell)}\bigr),
               \qquad \ell = L-1,\dots,1$$

So, we got for all parameters,
$$

\frac{\partial\mathcal L}{\partial W^{(\ell)}}= \delta^{(\ell)} \, \big(a^{(\ell-1)}\big)^\top$$
$$\frac{\partial\mathcal L}{\partial b^{(\ell)}}
  = \delta^{(\ell)}
$$


### Non convexity of ANN

The optimisation is not convexe in despite we have convex activation function.
In some case we can expect convex problem but it's too restrictive
> https://stats.stackexchange.com/questions/499120/are-there-any-convex-neural-networks  

> Input Convex Neural Networks  
Brandon Amos 1 Lei Xu 2 * J. Zico Kolter 1

### Optimizer

### Basic 

$$
W_{t+1} = W_t - \eta \nabla W_t
$$

In [4]:
# To keep the same syntax as below but i agree is too much for nothing

mutable struct SGD_scratch
end

function SGD_scratch(model::MLP)
    SGD_scratch()
end

function sgd_scratch_update!(model::MLP, grads, opt::SGD_scratch, η)
    @assert length(model.layers) == length(grads)
    for (L, (dW, db)) in zip(model.layers, grads)
        L.W .-= η .* dW          # every layer
        L.b .-= η .* db
    end
end

sgd_scratch_update! (generic function with 1 method)

#### Adam

So we will use the optimizer Adam because it works very well, $W_t$ is the gradient of weights and biases at the $t$-th epochs  
$\eta$ is the learing rate

$$m_0 = 0, \quad v_0 = 0, \quad 1 > \beta_1, \beta_2 \geq 0, \quad \epsilon >0$$
$$
   m_t = \beta_1 m_{t-1} + (1 - \beta_1) \partial W_t
$$
$$
   v_t = \beta_2 v_{t-1} + (1 - \beta_2) (\partial W_t)^2
$$
$$
   \hat{m}_t = \frac{m_t}{1 - \beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1 - \beta_2^t}
$$
$$
   W_{t+1} = W_t - \frac{\eta}{\sqrt{\hat{v}_t + \epsilon}} \hat{m}_t
$$

> https://arxiv.org/pdf/1412.6980   

> https://arxiv.org/pdf/1904.09237   

In [5]:
mutable struct Adam_scratch
    m::Vector{Matrix}; v::Vector{Matrix}
    mb::Vector{Vector}; vb::Vector{Vector}
    β1::Float64; β2::Float64; ε::Float64; t::Int
end

function Adam_scratch(model::MLP; β1=.9, β2=.999, ε=1e-8)
    L = length(model.layers)
    Adam_scratch([zeros(size(L.W)) for L in model.layers],
         [zeros(size(L.W)) for L in model.layers],
         [zeros(size(L.b)) for L in model.layers],
         [zeros(size(L.b)) for L in model.layers],
         β1, β2, ε, 0)
end

function adam_scratch_update!(model::MLP, grads, opt::Adam_scratch, η)
    opt.t += 1
    for (ℓ,(dW,db)) in enumerate(grads)
        L = model.layers[ℓ]

        # weights
        opt.m[ℓ] .= opt.β1 .* opt.m[ℓ] .+ (1-opt.β1).*dW
        opt.v[ℓ] .= opt.β2 .* opt.v[ℓ] .+ (1-opt.β2).* (dW.^2)
        m̂ = opt.m[ℓ] ./ (1 - opt.β1^opt.t)
        v̂ = opt.v[ℓ] ./ (1 - opt.β2^opt.t)
        L.W .-= η .* m̂ ./ (sqrt.(v̂) .+ opt.ε)

        # biases
        opt.mb[ℓ] .= opt.β1 .* opt.mb[ℓ] .+ (1-opt.β1).*db
        opt.vb[ℓ] .= opt.β2 .* opt.vb[ℓ] .+ (1-opt.β2).* (db.^2)
        m̂b = opt.mb[ℓ] ./ (1 - opt.β1^opt.t)
        v̂b = opt.vb[ℓ] ./ (1 - opt.β2^opt.t)
        L.b .-= η .* m̂b ./ (sqrt.(v̂b) .+ opt.ε)
    end
end

adam_scratch_update! (generic function with 1 method)

## Initialisation

There are good ways to initialize weight matrices and bias vectors depending on the activations functions.  
<span style="color:red"><b>TODO :</b></span> comment on initialise dans notre cas ? car c'est pas ReLU nous masi $ReLU_\alpha$

# From scratch

I define MLP at the start of the notebook because it allows to get Adam in the good section and execute all cells from the button "run all"

In [6]:
relu(x)  = max.(0, x)
drelu(x) = x .> 0

relu_shift(x; α=1.0) = max(0.0,  x - α)
drelu_shift(x; α=1.0) = x > α ? 1.0 : 0.0 

softplus(x)  = log1p(exp(x))
dsoftplus(x) = 1/(1+exp(-x))

elu(x; α=1.0)  = x ≥ 0 ? x : α*(exp(x)-1)
delu(x; α=1.0) = x ≥ 0 ? 1.0 : α*exp(x)

Identity(α=1.0) = (x) -> x
dIdentity(α=1.0) = (x) -> 1.0

dIdentity (generic function with 2 methods)

In [184]:
"""
    build_mlp(sizes::Vector{Int}, activations::Vector{Pair})

`activations[k] = σ=>σ′` for hidden layer k (1-based).  
The output layer is linear (regression linear).
"""
function build_mlp(sizes::Vector{Int},activations::Vector{<:Pair{<:Function,<:Function}};rng=Random.default_rng())
    Random.seed!(42)
    L = length(sizes)-1
    layers = Vector{Dense_scratch}(undef, L)
    for ℓ in 1:L
        n_in, n_out = sizes[ℓ], sizes[ℓ+1]
        # on each layer we take the corresponding ς and ς' functions except for the last layer 
        ς,ς′ = ℓ < L ? (activations[ℓ].first, activations[ℓ].second) : (identity, _->1.0)
        # scale = 1/sqrt(n_in) # TODO :  sqrt(2/n_in) mieux faire l'initialization (He/Kaiming)
        # scale =0
        scale = 1.
        # layers[ℓ] = Dense_scratch(scale*randn(rng,n_out,n_in), zeros(n_out), ς, ς′)
        layers[ℓ] = Dense_scratch(fill(1 / sqrt(n_in), n_out, n_in), zeros(n_out), ς, ς′)

    end
    return MLP(layers)
end

build_mlp

In [179]:
function forward(model::MLP, x)
    a = x
    for L in model.layers
        a = L.ς.(L.W*a .+ L.b)
    end
    return a
end

forward (generic function with 1 method)

In [9]:
"""
    Loss(loss_fn, delta_fn)

f  : (ŷ, y) → real scalar   (for logging / early–stop)
d_f : (ŷ, y) → δ⁽ᴸ⁾          (seed for back-prop)
"""
struct Loss{F,G}
    f :: F
    d_f :: G
end

Loss

In [10]:
"""
    backprop(model, x, y, loss′)

Input
`x` is a column vector,
`y` is the target output (column vector),
`loss′` is the gradient of the loss function at the output layer (column vector).
Returns `grads` : Vector of (dW, db) in layer-order.
"""
function backprop(model::MLP, x, y, loss::Loss)
    a = x
    A = [x]
    Z = Vector{Vector}()    # pre-activations
    for L in model.layers
        z = L.W*a .+ L.b
        push!(Z,z)

        a = L.ς.(z)
        push!(A,a)
    end

    δ = loss.d_f(A[end], y)
    Ltot = length(model.layers)
    grads = Vector{Tuple{Matrix,Vector}}(undef, Ltot)

    for ℓ in Ltot:-1:1
        L = model.layers[ℓ]

        a_prev = A[ℓ]
        δ = δ .* L.ς′.(Z[ℓ])
        grads[ℓ] = (δ*a_prev', δ)
        δ = L.W' * δ # propagation
    end
    return grads
end

backprop

In [11]:
function train!(model, X, y, opt, opt_update!, loss::Loss; tol=1e-6, epochs=200, batch=32, η=1e-3, verbose=false, print_freq=100)
    optimiser = opt(model)
    N = size(X,2)
    val_loss_prev = Inf
    for epoch in 1:epochs
        perm = randperm(N)
        for k in 1:batch:N
            idx = perm[k:min(k+batch-1, N)]
            grads_sum = nothing

            for j in idx
                grads = backprop(model, X[:,j], y[j], loss)

                if grads_sum === nothing
                    grads_sum = grads
                else
                    for ℓ in eachindex(grads)
                        grads_sum[ℓ][1] .+= grads[ℓ][1]
                        grads_sum[ℓ][2] .+= grads[ℓ][2]
                    end
                end
            end
            # average
            for ℓ in eachindex(grads_sum)
                grads_sum[ℓ][1] ./= length(idx)
                grads_sum[ℓ][2] ./= length(idx)
            end
            opt_update!(model, grads_sum, optimiser, η)
        end

        y_pred = forward(model, X)
        val_loss = loss.f(y_pred, y, model)

        if verbose && (epoch == 1 || epoch % print_freq == 0)
            @printf("Epoch %d/%d, Loss: %.6e\n", epoch, epochs, val_loss)
        end

        if abs(val_loss - val_loss_prev) < tol
            if verbose
                @printf("Converged at epoch %d, Loss: %.6e\n", epoch, val_loss)
            end
            break
        end
        val_loss_prev = val_loss
    end
end

train! (generic function with 1 method)

## Linear model (MSE)

In [12]:
Random.seed!(42)
n,p = 200,20
X = randn(n,p)
βtrue = randn(p) .* (rand(p) .< 0.3) # sparse
y = X*βtrue + 0.05*randn(n)
λ = 0. # lasso with lambda=0 is mse
;

In [13]:
sizes = [p,1] # zero hidden layer

mse_loss = Loss(
    (y_pred, y, model) -> 0.5 * sum((y_pred[1,:] .- y).^2), # flatten y, other option: vec(y_pred)
    (y_pred, y) -> y_pred .- y
)

net_sgd  = build_mlp(sizes, Pair{Function,Function}[]) # no activations
net_adam  = build_mlp(sizes,  Pair{Function,Function}[]) # no activations
train!(net_sgd, X', y, SGD_scratch, sgd_scratch_update!, mse_loss ;tol=1e-12, epochs=300, η=1/opnorm(X)^2, batch=size(X,2), verbose=true , print_freq=1000)
train!(net_adam, X', y, Adam_scratch, adam_scratch_update!, mse_loss ;tol=1e-12, epochs=300, η=1/opnorm(X)^2, batch=size(X,2), verbose=true , print_freq=1000)
;

Epoch 1/300, Loss: 1.074235e+03
Epoch 1/300, Loss: 1.108187e+03


In [14]:
βhat_nn_sgd = net_sgd.layers[1].W[:]
βhat_nn_adam = net_adam.layers[1].W[:]
;

In [15]:
f(b) = 0.5*norm(y-X*b,2)^2
∇f(b) = -X'*(y - X*b)
g(x) = λ*sum(abs, x) # L1 norm
prox_L(x, step) = sign.(x) .* max.(abs.(x) .- λ*step, zero(x))
λ = 0
beta0 = zeros(p)# initial conditions
βhat_ista = ista_L(beta0, f, g, ∇f, opnorm(X)^2, prox_L, tol=1e-12)
;

[ISTA] iter     1  cost=1.682e+02  diff=9.792e+02
[ISTA] END iter    51  cost=2.183e-01  diff=8.733e-13


In [16]:
@printf("\n||β̂_NN_sgd - β̂_ISTA||₂ = %.3e\n", norm(βhat_nn_sgd-βhat_ista))
@printf("\n||β̂_NN_adam - β̂_ISTA||₂ = %.3e\n", norm(βhat_nn_adam-βhat_ista))


||β̂_NN_sgd - β̂_ISTA||₂ = 1.670e-02

||β̂_NN_adam - β̂_ISTA||₂ = 7.557e-04


It seems to be working but I can't success to get a better vector (1e-4)

## Others tests

<span style="color:red"><b>TODO :</b></span> Mon code fonctionne mais c'est sur de petits détails qu'on remarque des différences donc j'ai pas d'idées pour voir si l'implémentation est bonne.

### 3-layers, ReLU, same data

This test is only to see if the output is exactly the same than from Flux.jl

In [49]:
Random.seed!(42)
n,p = 20,2
X = randn(n,p)
βtrue = randn(p) .* (rand(p) .< 0.3) # sparse
y = X*βtrue + 0.05*randn(n)
λ = 0. # lasso with lambda=0 is mse
;

In [50]:
sizes = [p,1,16,1] # zero hidden layer

mse_loss = Loss(
    (y_pred, y, model) -> 0.5 * sum((y_pred[1,:] .- y).^2), # flatten y, other option: vec(y_pred)
    (y_pred, y) -> y_pred .- y
)

net_sgd  = build_mlp(sizes, Pair{Function,Function}[
    relu_shift => drelu_shift,
    relu_shift => drelu_shift,
    Identity() => dIdentity()
])
net_adam  = build_mlp(sizes, Pair{Function,Function}[
    relu_shift => drelu_shift,
    relu_shift => drelu_shift,
    Identity() => dIdentity()
])

# ? J'ai baissé le nombre d'epochs pour que ça aille plsu vite mais mettez 30_000
train!(net_sgd, X', y, SGD_scratch, sgd_scratch_update!, mse_loss ;tol=1e-12, epochs=30, η=1/opnorm(X)^2, batch=size(X,2), verbose=true , print_freq=1000)
train!(net_adam, X', y, Adam_scratch, adam_scratch_update!, mse_loss ;tol=1e-12, epochs=30, η=1/opnorm(X)^2, batch=size(X,2), verbose=true , print_freq=1000)
;

Epoch 1/30, Loss: 2.266242e-02
Epoch 1/30, Loss: 2.452424e-02


In [51]:
net_sgd.layers[1].W

1×2 Matrix{Float64}:
 0.0  0.0

In [52]:
βhat_nn_sgd = net_sgd.layers[1].W[:]
y_pred_sgd = net_sgd.layers[1].W * X' .+ net_sgd.layers[1].b

βhat_nn_adam = net_adam.layers[1].W[:]
y_pred_adam = net_adam.layers[1].W * X' .+ net_adam.layers[1].b
;

# From Julia

In [134]:
using Lux, Optimisers, MLUtils, Functors, Zygote, NNlib, Statistics

## Linear model (MSE)

I did only **ADAM** but **SGD** works also as precdent experiment !

### 3-layers, ReLU, same data

In [135]:
Random.seed!(42)
n,p = 20,2
X = randn(n,p)
βtrue = randn(p) .* (rand(p) .< 0.3) # sparse
y = X*βtrue + 0.05*randn(n)
λ = 0. # lasso with lambda=0 is mse
;

#### Adam

In [136]:
# TODO : faire un warm start (initialiser la compilation, sinon trop long)
# ---------------- synthetic data ----------------
Random.seed!(42)
# n, p = 200, 20 # valeurs que j'ai utilisé
n, p = 20,2
X     = randn(Float32, n, p)          # (obs, features)
βtrue = randn(Float32, p) .* (rand(Float32, p) .< 0.3f0)
y     = X * βtrue .+ 0.05f0 * randn(Float32, n)  # (obs,)
λ     = 0f0                             # LASSO penalty (mse if λ = 0)

X′  = permutedims(X)                    # Lux/Flux want (features, batch)
y′  = reshape(y, 1, :)

# ---------------- model ----------------
relu_shift(x) = NNlib.relu.(x .+ 1f-8)

# TODO : we have to use_bias !
model = Chain(
    Dense(p => 16, relu_shift;   use_bias = false),
    Dense(16 => 16, relu_shift;  use_bias = false),
    Dense(16 => 1,  identity;    use_bias = false)
)     

# ---------------- parameters & optimiser ----------------
η          = 1f0 / opnorm(X)^2          # learning-rate heuristic
rng        = Random.default_rng()
ps, st     = Lux.setup(rng, model)

opt        = Optimisers.Adam(η)

# ---------------- helpers ----------------
function l1(ps)                         # ∑|θ| over all leaves
    s = 0f0
    Functors.fmap(x -> x isa AbstractArray ? (s += sum(abs, x); x) : x, ps)
    return s
end

function loss(model, ps, st, (x, y))
    ŷ, st2 = model(x, ps, st)
    mse    = 0.5f0 * sum((ŷ .- y).^2) / size(y,2)
    return mse + λ*l1(ps), st2, (; mse)
end

# ---------------- data loader ----------------
batchsize    = 32
loader       = MLUtils.DataLoader((X′, y′); batchsize, shuffle = true)

┌ Warning: Number of observations less than batch-size, decreasing the batch-size to 20
└ @ MLUtils C:\Users\Le R\.julia\packages\MLUtils\5jDrc\src\batchview.jl:104


1-element DataLoader(::Tuple{Matrix{Float32}, Matrix{Float32}}, shuffle=true, batchsize=32)
  with first element:
  (2×20 Matrix{Float32}, 1×20 Matrix{Float32},)

In [137]:
# ---------------- training loop ----------------
backend      = Lux.AutoZygote()
ts           = Lux.Training.TrainState(model, ps, st, opt) 
nepochs      = 30_000
tol          = 1e-6
prev_loss    = Inf

@info "Starting training…"
for epoch in 1:nepochs
    curr = 0f0
    for batch in loader
        _, curr, _, ts = Lux.Training.single_train_step!(backend, loss, batch, ts;
                                                        return_gradients = false)
    end
    if abs(curr - prev_loss) < tol
        @info "Converged at epoch $epoch | loss = $(round(curr; sigdigits = 5))"
        break
    end
    if epoch % 100 == 0
        @info "Epoch $epoch | loss = $(round(curr; sigdigits = 5))"
    end
    prev_loss = curr
end


┌ Info: Starting training…
└ @ Main c:\Users\Le R\Desktop\Code\Projets\Geneve\STAGE-GENEVE\RAPHAEL\notebook_explain\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y105sZmlsZQ==.jl:8
┌ Info: Converged at epoch 26 | loss = 0.0006658
└ @ Main c:\Users\Le R\Desktop\Code\Projets\Geneve\STAGE-GENEVE\RAPHAEL\notebook_explain\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y105sZmlsZQ==.jl:16


In [57]:
W1 = ps.layer_1

(weight = Float32[0.85815114 -0.31487662; -0.28055164 0.31933126; … ; -0.16486089 0.30668002; 0.80982757 0.5465371],)

In [58]:
W1 = ps.layer_1.weight      # first layer weight   (Float32 matrix)

16×2 Matrix{Float32}:
  0.858151   -0.314877
 -0.280552    0.319331
  0.678927    0.06178
  0.68123    -0.389811
 -0.0627615  -0.0377295
 -0.273092   -0.52679
  0.300346   -0.729362
 -0.587113   -0.504232
 -0.929384   -0.199617
 -0.481189   -0.510561
  0.0895388   0.568804
 -0.366615   -0.379062
 -0.1682     -0.393172
 -0.694414   -0.824898
 -0.164861    0.30668
  0.809828    0.546537

In [59]:
# ---------------- fitted coefficients / predictions ----------------
ŷ, _      = model(X′, ts.parameters, ts.states)
y_pred_lux_adam  = ŷ[:] # analogue of `βhat_flux_adam`


20-element Vector{Float32}:
 -0.015817653
  0.0
  0.0
  0.0
  0.0
  0.0018295145
  0.0
 -0.0039392537
  0.027774425
  0.0008137302
 -0.01563457
  0.0
  0.0
  0.008989939
 -0.002883925
 -0.0027699831
  0.0012075545
  0.0
 -0.021333545
 -0.0031455518

In [60]:
# @printf("\n norm(y_pred_adma -y_prd_lux_adma))
@printf("\n||y_pred_adam - y_pred_lux_adam||₂ = %.3e\n", norm(vec(y_pred_adam) - y_pred_lux_adam))
@printf("\n||y_pred_sgd - y_pred_lux_adam||₂ = %.3e\n", norm(vec(y_pred_sgd) - y_pred_lux_adam))

# @printf("\n||β̂_NN_adam - β̂_lux_adam||₂ = %.3e\n", norm(βhat_nn_adam-βhat_lux))


||y_pred_adam - y_pred_lux_adam||₂ = 4.300e-02

||y_pred_sgd - y_pred_lux_adam||₂ = 4.300e-02


# Comparisons

In [61]:
include("../../comp_translate/SaveLoadTxt.jl") # Assuming you have a SaveLoadTxt.jl file with the necessary functions
using .SaveLoadTxt

## Data

### Non linear

In [150]:
#
n = 20
p = 50
s = 5
# Random.seed!(42)
X = randn(n, p)
# y = cos(x_1) + x_2^2 - x_3 + exp(x_4) - x_5 + 0.1 * N(0,1)
y = cos.(X[:,1]) .+ X[:,2].^2 .- X[:,3] .+ exp.(X[:,4]) .- X[:,5] .+ 0.1 * randn(n)
layers = [p,3,1]
;

In [151]:
dump_txt("../../comp_translate/data/ANN_non_linear_sparse.txt", [X, y, layers]; append = false)
loaded = load_txt("../../comp_translate/data/ANN_non_linear_sparse.txt")
# @assert loaded == [X,y, layers]
println("Round-trip OK ")

Round-trip OK 


## Results 

+ SGD & Adam & ISTA
+ Initialisation : TODO
+ Loss : 0.5 * MSE
+ Batchsize : all data
+ ReLU activations functions

PLus tard : 
+ Penalty : $\ell_1$

On considere 0.5*MSE pour utilsier ISTA sans backtracking

### SGD

#### Scratch

In [226]:
sizes = loaded[3]

mse_loss = Loss(
    (y_pred, y, model) -> 0.5 * sum((y_pred[1,:] .- y).^2)/ size(y,1), # flatten y, other option: vec(y_pred)
    (y_pred, y) -> (y_pred .- y)/ size(y,1)
)

# Relu activaitosn
net_sgd = build_mlp(sizes,
    Pair{Function,Function}[
        # (x -> relu_shift(x; α=0.0)) => (x -> drelu_shift(x; α=0.0)),
        relu => drelu,
        Identity => (x->ones(size(x)))
    ])
train!(net_sgd, X', y, SGD_scratch, sgd_scratch_update!, mse_loss ;tol=1e-12, epochs=50_000, η=1/opnorm(X)^2, batch=size(X,2), verbose=true , print_freq=100)
;

Epoch 1/50000, Loss: 6.331675e+00
Epoch 100/50000, Loss: 3.437376e+00
Epoch 200/50000, Loss: 3.210058e+00
Epoch 300/50000, Loss: 3.099914e+00
Epoch 400/50000, Loss: 3.050248e+00
Epoch 500/50000, Loss: 3.035163e+00
Epoch 600/50000, Loss: 3.028725e+00
Epoch 700/50000, Loss: 3.025806e+00
Epoch 800/50000, Loss: 3.024462e+00
Epoch 900/50000, Loss: 3.023839e+00
Epoch 1000/50000, Loss: 3.023550e+00
Epoch 1100/50000, Loss: 3.023416e+00
Epoch 1200/50000, Loss: 3.023354e+00
Epoch 1300/50000, Loss: 3.023324e+00
Epoch 1400/50000, Loss: 3.023311e+00
Epoch 1500/50000, Loss: 3.023305e+00
Epoch 1600/50000, Loss: 3.023302e+00
Epoch 1700/50000, Loss: 3.023300e+00
Epoch 1800/50000, Loss: 3.023300e+00
Epoch 1900/50000, Loss: 3.023299e+00
Epoch 2000/50000, Loss: 3.023299e+00
Epoch 2100/50000, Loss: 3.023299e+00
Epoch 2200/50000, Loss: 3.023299e+00
Epoch 2300/50000, Loss: 3.023299e+00
Epoch 2400/50000, Loss: 3.023299e+00
Epoch 2500/50000, Loss: 3.023299e+00
Epoch 2600/50000, Loss: 3.023299e+00
Epoch 2700/50

In [227]:
y_p_sc_sgd_mse = forward(net_sgd, X')[:]
βhat_sc_sgd_mse = net_sgd.layers[1].W[:];

#### Lux

In [228]:
relu_shift(x) = NNlib.relu.(x)
# σ = 1/sqrt(size(X,1))            # same std as scratch
σ = 1.
rng = Random.default_rng()

initW(rng, out, in) = fill(1 / sqrt(in), out, in)
initb(rng, out)     = zeros(Float32, out)

model = Chain(
    Dense(p => 3, relu_shift;  use_bias = true, init_weight = initW, init_bias = initb),
    Dense(3 => 1, identity; use_bias = true, init_weight = initW, init_bias = initb)
)

η = 1f0 / opnorm(X)^2 # learning-rate heuristic

ps, st = Lux.setup(rng, model)

# opt = Optimisers.Adam(η)
opt = Optimisers.Descent(η)

# function l1(ps)                         # ∑|θ| over all leaves
#     s = 0f0
#     Functors.fmap(x -> x isa AbstractArray ? (s += sum(abs, x); x) : x, ps)
#     return s
# end

function loss(model, ps, st, (x, y))
    ŷ, st2 = model(x, ps, st)
    mse    = 0.5f0 * sum((ŷ .- y).^2) / size(y,2)
    return mse , st2, (; mse)
end

y′  = reshape(y, 1, :)
batchsize    = size(X,2)
loader       = MLUtils.DataLoader((X', y′); batchsize, shuffle = true)

┌ Warning: Number of observations less than batch-size, decreasing the batch-size to 20
└ @ MLUtils C:\Users\Le R\.julia\packages\MLUtils\5jDrc\src\batchview.jl:104


1-element DataLoader(::Tuple{Adjoint{Float64, Matrix{Float64}}, Matrix{Float64}}, shuffle=true, batchsize=50)
  with first element:
  (50×20 Matrix{Float64}, 1×20 Matrix{Float64},)

In [229]:
backend      = Lux.AutoZygote()
ts           = Lux.Training.TrainState(model, ps, st, opt) 
nepochs      = 30_000
tol          = 1e-12
prev_loss    = Inf

@info "Starting training…"
for epoch in 1:nepochs
    curr = 0f0
    for batch in loader
        _, curr, _, ts = Lux.Training.single_train_step!(backend, loss, batch, ts;return_gradients = false)
    end
    if abs(curr - prev_loss) < tol
        @info "Converged at epoch $epoch | loss = $(round(curr; sigdigits = 5))"
        break
    end
    if epoch % 100 == 0 || epoch==1
        @info "Epoch $epoch | loss = $(round(curr; sigdigits = 5))"
    end
    prev_loss = curr
end


┌ Info: Starting training…
└ @ Main c:\Users\Le R\Desktop\Code\Projets\Geneve\STAGE-GENEVE\RAPHAEL\notebook_explain\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y133sZmlsZQ==.jl:7
┌ Info: Epoch 1 | loss = 6.4456
└ @ Main c:\Users\Le R\Desktop\Code\Projets\Geneve\STAGE-GENEVE\RAPHAEL\notebook_explain\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y133sZmlsZQ==.jl:18
┌ Info: Epoch 100 | loss = 3.441
└ @ Main c:\Users\Le R\Desktop\Code\Projets\Geneve\STAGE-GENEVE\RAPHAEL\notebook_explain\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y133sZmlsZQ==.jl:18
┌ Info: Epoch 200 | loss = 3.2115
└ @ Main c:\Users\Le R\Desktop\Code\Projets\Geneve\STAGE-GENEVE\RAPHAEL\notebook_explain\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y133sZmlsZQ==.jl:18
┌ Info: Epoch 300 | loss = 3.1011
└ @ Main c:\Users\Le R\Desktop\Code\Projets\Geneve\STAGE-GENEVE\RAPHAEL\notebook_explain\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y133sZmlsZQ==.jl:18
┌ Info: Epoch 400 | loss = 3.0505
└ @ Main c:\

In [230]:
y_p_lux_sgd_mse, _ = model(X', ts.parameters, ts.states)
βhat_lux_sgd_mse = ts.parameters.layer_1.weight[:]

150-element Vector{Float64}:
  0.15325549524848178
  0.15325549524848178
  0.15325549524848178
  0.1409315851763199
  0.1409315851763199
  0.1409315851763199
 -0.03737682408715094
 -0.03737682408715094
 -0.03737682408715094
  0.12473458875783115
  ⋮
  0.2508999402405783
  0.2508999402405783
  0.2508999402405783
  0.1019488305179242
  0.1019488305179242
  0.1019488305179242
  0.15083442841144232
  0.15083442841144232
  0.15083442841144232

In [231]:
println("||y_p_sc_sgd_mse - y_p_lux_sgd_mse||₂ = ", norm(vec(y_p_sc_sgd_mse) - vec(y_p_lux_sgd_mse)))
println("||βhat_sc_sgd_mse - βhat_lux_sgd_mse||₂ = ", norm(βhat_sc_sgd_mse - βhat_lux_sgd_mse))

||y_p_sc_sgd_mse - y_p_lux_sgd_mse||₂ = 1.2819273875945559e-5
||βhat_sc_sgd_mse - βhat_lux_sgd_mse||₂ = 8.239194602466951e-7


Parfait ça fonctionne. On regarde avec Adam à présent

### Adam

#### Scratch

In [235]:
mse_loss = Loss(
    (y_pred, y, model) -> 0.5 * sum((y_pred[1,:] .- y).^2)/ size(y,1), # flatten y, other option: vec(y_pred)
    (y_pred, y) -> (y_pred .- y)/ size(y,1)
)

# Relu activaitosn
net_adam = build_mlp(sizes,
    Pair{Function,Function}[
        # (x -> relu_shift(x; α=0.0)) => (x -> drelu_shift(x; α=0.0)),
        relu => drelu,
        Identity => (x->ones(size(x)))
    ])
train!(net_adam, X', y, Adam_scratch, adam_scratch_update!, mse_loss ;tol=1e-12, epochs=3000, η=1/opnorm(X)^2, batch=size(X,2), verbose=true , print_freq=100)
;

Epoch 1/3000, Loss: 6.194063e+00
Epoch 100/3000, Loss: 2.586303e+00
Epoch 200/3000, Loss: 2.443698e+00
Epoch 300/3000, Loss: 2.386096e+00
Epoch 400/3000, Loss: 2.369109e+00
Epoch 500/3000, Loss: 2.365720e+00
Epoch 600/3000, Loss: 2.365283e+00
Epoch 700/3000, Loss: 2.365243e+00
Epoch 800/3000, Loss: 2.365241e+00
Epoch 900/3000, Loss: 2.365240e+00
Epoch 1000/3000, Loss: 2.365240e+00
Converged at epoch 1033, Loss: 2.365240e+00


In [236]:
y_p_sc_adam_mse = forward(net_adam, X')[:]
βhat_sc_adam_mse = net_adam.layers[1].W[:];

#### Lux

In [237]:
model = Chain(
    Dense(p => 3, relu_shift;  use_bias = true, init_weight = initW, init_bias = initb),
    Dense(3 => 1, identity; use_bias = true, init_weight = initW, init_bias = initb)
)
ps, st = Lux.setup(rng, model)

opt = Optimisers.Adam(η)

function loss(model, ps, st, (x, y))
    ŷ, st2 = model(x, ps, st)
    mse    = 0.5f0 * sum((ŷ .- y).^2) / size(y,2)
    return mse , st2, (; mse)
end

y′  = reshape(y, 1, :)
loader       = MLUtils.DataLoader((X', y′); batchsize, shuffle = true)

┌ Warning: Number of observations less than batch-size, decreasing the batch-size to 20
└ @ MLUtils C:\Users\Le R\.julia\packages\MLUtils\5jDrc\src\batchview.jl:104


1-element DataLoader(::Tuple{Adjoint{Float64, Matrix{Float64}}, Matrix{Float64}}, shuffle=true, batchsize=50)
  with first element:
  (50×20 Matrix{Float64}, 1×20 Matrix{Float64},)

In [238]:
backend      = Lux.AutoZygote()
ts           = Lux.Training.TrainState(model, ps, st, opt) 
nepochs      = 30_000
tol          = 1e-12
prev_loss    = Inf

@info "Starting training…"
for epoch in 1:nepochs
    curr = 0f0
    for batch in loader
        _, curr, _, ts = Lux.Training.single_train_step!(backend, loss, batch, ts;return_gradients = false)
    end
    if abs(curr - prev_loss) < tol
        @info "Converged at epoch $epoch | loss = $(round(curr; sigdigits = 5))"
        break
    end
    if epoch % 100 == 0 || epoch==1
        @info "Epoch $epoch | loss = $(round(curr; sigdigits = 5))"
    end
    prev_loss = curr
end


┌ Info: Starting training…
└ @ Main c:\Users\Le R\Desktop\Code\Projets\Geneve\STAGE-GENEVE\RAPHAEL\notebook_explain\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y150sZmlsZQ==.jl:7
┌ Info: Epoch 1 | loss = 6.4456
└ @ Main c:\Users\Le R\Desktop\Code\Projets\Geneve\STAGE-GENEVE\RAPHAEL\notebook_explain\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y150sZmlsZQ==.jl:18
┌ Info: Epoch 100 | loss = 2.5883
└ @ Main c:\Users\Le R\Desktop\Code\Projets\Geneve\STAGE-GENEVE\RAPHAEL\notebook_explain\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y150sZmlsZQ==.jl:18
┌ Info: Epoch 200 | loss = 2.4446
└ @ Main c:\Users\Le R\Desktop\Code\Projets\Geneve\STAGE-GENEVE\RAPHAEL\notebook_explain\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y150sZmlsZQ==.jl:18
┌ Info: Epoch 300 | loss = 2.3864
└ @ Main c:\Users\Le R\Desktop\Code\Projets\Geneve\STAGE-GENEVE\RAPHAEL\notebook_explain\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y150sZmlsZQ==.jl:18
┌ Info: Epoch 400 | loss = 2.3692
└ @ Main c:

In [239]:
y_p_lux_adam_mse, _ = model(X', ts.parameters, ts.states)
βhat_lux_adam_mse = ts.parameters.layer_1.weight[:]

150-element Vector{Float64}:
  0.23446048784984852
  0.23446048784984852
  0.23446048784984852
  0.3030017295803509
  0.3030017295803509
  0.3030017295803509
 -0.02757013009242761
 -0.02757013009242761
 -0.02757013009242761
  0.06496048646716612
  ⋮
  0.2872589202789973
  0.2872589202789973
  0.2872589202789973
  0.054689801470101655
  0.054689801470101655
  0.054689801470101655
  0.2531694380556684
  0.2531694380556684
  0.2531694380556684

In [241]:
# println(norm(vec(y_p_sc_sgd_mse) - vec(y_p_lux_adam_mse))
println("||y_p_sc_adam_mse - y_p_lux_adam_mse||₂ = ", norm(vec(y_p_sc_adam_mse) - vec(y_p_lux_adam_mse)))
println("||βhat_sc_adam_mse - βhat_lux_adam_mse||₂ = ", norm(βhat_sc_adam_mse - βhat_lux_adam_mse))

||y_p_sc_adam_mse - y_p_lux_adam_mse||₂ = 4.454340912653943e-6
||βhat_sc_adam_mse - βhat_lux_adam_mse||₂ = 4.366207290469105e-7
